<a href="https://colab.research.google.com/github/GenoaroSJr/DataScienceSmarkio/blob/main/Teste_musica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando dados

In [42]:
from google.colab import drive
import pandas as pd
import numpy as np
from	sklearn.model_selection	import	cross_val_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

In [28]:
drive.mount("/content/drive/")
df = pd.read_excel("/content/drive/MyDrive/musica_dados.xlsx")
df.head()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


,letra,artista
0,Jay-z Uh-uh-uh You ready b? Let's go get 'em. ...,Beyoncé
1,Your challengers are a young group from Housto...,Beyoncé
2,"Dum-da-de-da Do, do, do, do, do, do (Coming do...",Beyoncé
3,If I ain't got nothing I got you If I ain't go...,Beyoncé
4,Six inch heels She walked in the club like nob...,Beyoncé


# Tratando dados

In [29]:
#Separando os artistas
artista = df['artista']

In [30]:
#Se necessários, fazer com que os dados categóricos se tornem numéricos
#for i in range(len(df)):
#  if (artista[i] == 'Beyoncé'):
#    artista[i] = 1
#  else:
#    artista[i] = 2
#print(len(df))
#df.head(500)

In [31]:
#Pegando os textos dcoluna "letra"
textos = df['letra']

In [32]:
#Pegando cada palavra de forma isolada
palavrasIsoladas = textos.str.lower().str.split()
print(palavrasIsoladas)

0      [jay-z, uh-uh-uh, you, ready, b?, let's, go, g...
1      [your, challengers, are, a, young, group, from...
2      [dum-da-de-da, do,, do,, do,, do,, do,, do, (c...
3      [if, i, ain't, got, nothing, i, got, you, if, ...
4      [six, inch, heels, she, walked, in, the, club,...
                             ...                        
513    [yeah, yeah, yeah, yeah, i, ain't, trying, to,...
514    [you, the, one, that, i, dream, about, all, da...
515    [no,, no,, no, you, don't, love, me, and, i, k...
516    [you, should, be, mine, oh, baby,, oh, baby,, ...
517    [[rihanna], i, remember, when, the, world, was...
Name: letra, Length: 518, dtype: object


### Criando dicionário

In [33]:
#pegando tadas as palavras que aparecem nos textos
dicionario = set()

for lista in palavrasIsoladas:
  dicionario.update(lista)

#print(dicionario)
len(dicionario)

{'wait.', 'affect', 'climbling', 'song)', 'reste', 'hoy...', '"baby,', 'thots', 'lenny', 'blonder', 'pitiful', 'rocket', 'src="/img/etc/instrumental.png"/>', 'range,', 'customers', 'cuento', 'gucci', 'seyfried', '(ewww)', 'stayed', '(even', 'sanging', 'ashanti:', 'waste', '(why)', 'youte),', 'off!', 'true...', 'cast', 'state,', 'talking', "lyin',", 'ho,', 'navy', 'wicked', 'expected', 'nightime', 'rainy', 'hit', 'phrase', 'spot,', 'taught', 'na-na-na-na', "dancin'", 'cookin,', 'ohhha', 'acting', 'f-ck-ng', 'me.', 'want!', 'sweats', 'explosent', 'cruise', "breathin'", '(raise', 'satisfy', 'weaker', 'partent', 'ayo,', 'scratched', '(hoooo!)', "heartbreak's", 'affectionate', 'okay,', '(lil', 'sangria', 'plow', 'ward', "'cross", 'vivía', 'faite,', "smokin'", 'warned', 'ships,', 'kaled', 'year', 'dunce', 'express', 'some!', 'that...', 'nest', "swangin'", 'babymomma', 'achetant', 'rich', 'justice', 'white,', 'bleed', 'messed', 'sumo', 'personal', 'overtake', 'woulda', 'allez', 'suckas', 'fli

11726

### Vinculando palavras a posição em um vetor:

In [34]:
totalDePalavras = len(dicionario)
palavraEposicao = dict(zip(dicionario, range(totalDePalavras)))
#print(palavraEposicao)

{'wait.': 0, 'affect': 1, 'climbling': 2, 'song)': 3, 'reste': 4, 'hoy...': 5, '"baby,': 6, 'thots': 7, 'lenny': 8, 'blonder': 9, 'pitiful': 10, 'rocket': 11, 'src="/img/etc/instrumental.png"/>': 12, 'range,': 13, 'customers': 14, 'cuento': 15, 'gucci': 16, 'seyfried': 17, '(ewww)': 18, 'stayed': 19, '(even': 20, 'sanging': 21, 'ashanti:': 22, 'waste': 23, '(why)': 24, 'youte),': 25, 'off!': 26, 'true...': 27, 'cast': 28, 'state,': 29, 'talking': 30, "lyin',": 31, 'ho,': 32, 'navy': 33, 'wicked': 34, 'expected': 35, 'nightime': 36, 'rainy': 37, 'hit': 38, 'phrase': 39, 'spot,': 40, 'taught': 41, 'na-na-na-na': 42, "dancin'": 43, 'cookin,': 44, 'ohhha': 45, 'acting': 46, 'f-ck-ng': 47, 'me.': 48, 'want!': 49, 'sweats': 50, 'explosent': 51, 'cruise': 52, "breathin'": 53, '(raise': 54, 'satisfy': 55, 'weaker': 56, 'partent': 57, 'ayo,': 58, 'scratched': 59, '(hoooo!)': 60, "heartbreak's": 61, 'affectionate': 62, 'okay,': 63, '(lil': 64, 'sangria': 65, 'plow': 66, 'ward': 67, "'cross": 68,

### Contando a presença de cada palavra:

> Para cada frase, o algoritmo irá retornar {x, y, z} mostrando quais palavras do dicionário aparece em determinada frase, sendo x, y e z a quantidade

In [35]:
def vetorizarPresencaPalavras(texto, palavraEposicao):
  vetor = [0] * len(palavraEposicao)
  for palavra in texto:
    if palavra in palavraEposicao:
      posicao = palavraEposicao[palavra]
      vetor[posicao] += 1
  return vetor

vetoresDeTexto = [vetorizarPresencaPalavras(texto, palavraEposicao) for texto in palavrasIsoladas]
#print(vetoresDeTexto)

In [36]:
# x - classificador
# y - classe

x = np.array(vetoresDeTexto)
y = np.array(artista)

porcentagem = 0.8

In [37]:
#definindo tamanho treino e validação
tamanho_treino = int(porcentagem*len(y))
tamanho_validacao = len(y) - tamanho_treino

In [38]:
#dados de treino
treino_dados = x[0:tamanho_treino]
treino_marcacoes = y[0:tamanho_treino]

In [39]:
#dados de validacao
validacao_dados = x[tamanho_treino:]
validacao_marcacoes = y[tamanho_treino:]

In [49]:
#Criando a extrutura para treinamento 
def predict(nome, modelo, treino_dados, treino_marcacoes):
    k = 10
    scores = cross_val_score(modelo, treino_dados, treino_marcacoes, cv = k)
    taxa_de_acerto = np.mean(scores)
    msg = "Taxa de acerto do {0}: {1}".format(nome, taxa_de_acerto)
    print(msg)

In [55]:
#Utilizando o classificador OneVsRest
modeloOneVsRest = OneVsRestClassifier(LinearSVC(random_state = 0, max_iter=10000))
resultadoOneVsRest = predict("OneVsRest", modeloOneVsRest, treino_dados, treino_marcacoes)

Taxa de acerto do OneVsRest: 0.6112078977932637


> O algoritmo foi escrito dessa forma para que, se necessário, possa ser integrado um outro algoritmo de ML e comparar os valors.